# testConditionalGAN1

Andrew E. Davidson
aedavids@ucsc.edu
8/29/24

Copyright (c) 2020-2023, Regents of the University of California All rights reserved. 
https://polyformproject.org/licenses/noncommercial/1.0.0  

AIM: create a simple GAN that is easy to test our basic framework  

generate y = x^2

ref: chapter 6. in Generative Advisarial Networks with Python

In [1]:
import ipynbname

from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from matplotlib import pyplot

notebookName = ipynbname.name()
notebookPath = ipynbname.path()
notebookDir = os.path.dirname(notebookPath)

outDir = f'{notebookDir}/{notebookName}.out'
imgOut = f'{outDir}/img'
print(f'imgOut:\n{imgOut}')

2024-10-14 16:43:13.996091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


imgOut:
/private/home/aedavids/extraCellularRNA/intraExtraRNA_POC/jupyterNotebooks/elife/gan/testConditionalGAN1.out/img


In [2]:
import tensorflow
print('tensorflow: %s' % tensorflow.__version__)
import keras
print('keras: %s' % keras.__version__)

tensorflow: 2.17.0
keras: 3.6.0


## define models

In [3]:
# define the standalone discriminator model
def defineDiscriminator(n_inputs=2):
    '''
    the discriminator is a binary classifier. ie real or fake
    if real y = 1
    if fake y = 0
    '''
    model = Sequential(name="discriminator")
    model.add( Input( shape=(n_inputs,)) )
    # model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            keras.metrics.BinaryAccuracy(),
            keras.metrics.FalseNegatives(),
        ],
    )
    
    return model

In [4]:
# define the standalone generator model
def defineGenerator(latent_dim, n_outputs=2):
    model = Sequential(name="generator")
    model.add( Input(shape=(latent_dim,)) )
    # model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(15, activation='relu', kernel_initializer='he_uniform'))
    # activation is linear. ie we want to generate real numbers
    model.add(Dense(n_outputs, activation='linear'))

    # notice we do not compile the generator
    
    return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def defineGan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    
    # connect them
    model = Sequential(name="GAN")
    
    # AEDWIP ???model.add( Input(shape=(1,)) )
    
    # add generator
    model.add(generator)
    
    # add the discriminator
    model.add(discriminator)
    
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model

## define data utility functions

In [6]:
# generate n real samples with class labels
def generateRealSamples(n):
    '''
    first col of X is x
    the second col of X is x^2
    
    y = 1 for real samples
    '''
    # generate inputs in [-0.5, 0.5]
    X1 = rand(n) - 0.5
    
    # generate outputs X^2
    X2 = X1 * X1
    
    # stack arrays
    X1 = X1.reshape(n, 1)
    X2 = X2.reshape(n, 1)
    X = hstack((X1, X2))
    
    # generate class labels
    y = ones((n, 1))
    
    return X, y

In [7]:
# generate points in latent space as input for the generator
def generateLatentPoints(latent_dim, n):
    '''
    generate noise vector. noice is normally distributed.
    '''
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    
    return x_input

In [8]:
# use the generator to generate n fake examples, with class labels
def generateFakeSamples(generator, latent_dim, n):
    '''
    y = 0 for fake samples
    '''
    # generate points in latent space
    x_input = generateLatentPoints(latent_dim, n)
    
    # predict outputs
    X = generator.predict(x_input)
    
    # create class labels
    y = zeros((n, 1))
    
    return X, y

## debug

In [9]:
def printLastLayerWeights( model ) :
    last_layer = model.layers[-1]
    weights = last_layer.get_weights()

    print(f'modelName : {model.name} last layer weights :\n{weights}')

In [10]:
def generateRealTrainingExamples(batchSize, debug=False):
    x_real, y_real = generateRealSamples(batchSize)
    if debug:
        print(f'x_real :\n{x_real} ')
        print(f'y_real :\n{y_real}')

    return (x_real, y_real)

In [11]:
def generateFakeTrainingExamples(generator, latent_dim, batchSize, debug=False):
    # prepare fake examples
    x_fake, y_fake = generateFakeSamples(generator, latent_dim, batchSize)

    if debug:
        print(f'x_fake :\n{x_fake}')
        print(f'y_fake :\n{y_fake}')    

    return (x_fake, y_fake)

In [15]:
def debugGAN():

    # create the discriminator
    discriminator = defineDiscriminator()
    discriminator.summary()
    
    # create the generator
    latent_dim = 5    
    generator = defineGenerator(latent_dim)
    print()
    generator.summary()
    
    #create the gan
    gan_model = defineGan(generator, discriminator)
    print()
    gan_model.summary()
    print()

    
    halfBatchSize = 3
    batchSize = halfBatchSize * 2
    x_real, y_real = generateRealTrainingExamples(halfBatchSize)
    x_fake, y_fake = generateFakeTrainingExamples(generator, latent_dim, halfBatchSize)

    print(f'\n################# test discriminator train_on_batch')
    print(f'after discriminator.train_on_batch()')
    printLastLayerWeights( discriminator )

    # when we create the GAN model trainable was set to FALSE
    # tran_on_batch() AttributeError: 'NoneType' object has no attribute 'update_state'
    # discriminator.trainable = True     
    # discriminator.train_on_batch(x_real, y_real)
    # print(f'\nafter discriminator.train_on_batch()')
    # printLastLayerWeights( discriminator )



    # print(f'\n################# after create GAN : test discriminator train_on_batch')
    # # AssertionError: Called a function referencing variables which have been deleted. This likely means that function-local variables were created and not referenced elsewhere in the program. This is generally a mistake; consider storing variables in an object attribute on first call.
    # discriminator.trainable = True     

    # x_real, y_real = generateRealTrainingExamples(half_batch)
    # x_fake, y_fake = generateFakeTrainingExamples(generator, latent_dim, half_batch)

    # discriminator.train_on_batch(x_real, y_real)
    # print(f'after discriminator.train_on_batch()')
    # printLastLayerWeights( discriminator )    

    print(f'before gan.train_on_batch()')    
    # prepare points in latent space as input for the generator
    x_gan = generateLatentPoints(latent_dim, batchSize)
    
    # create inverted labels for the fake samples
    # we want the discriminator to think the fakes are real
    y_gan = ones((batchSize, 1))
    
    # update the generator via the discriminator's error
    gan_model.train_on_batch(x_gan, y_gan) # AEDWIP return_dict=False
    print(f'\n gan_model.train_on_batch dis last layer weight')
    printLastLayerWeights( discriminator )

    print(f'\n gan_model.train_on_batch gan last layer weight')
    printLastLayerWeights( gan_model )

    # print(f'\n################# test GAN train_on_batch')


debugGAN()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 25)             │            75 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ generator (Sequential)          │ (None, 2)              │           122 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ discriminator (Sequential)      │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223 (892.00 B)

 Trainable params: 122 (488.00 B)

 Non-trainable params: 101 (404.00 B)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

################# test discriminator train_on_batch
after discriminator.train_on_batch()
modelName : discriminator last layer weights :
[array([[ 0.4115125 ],
       [ 0.0590893 ],
       [ 0.3830902 ],
       [ 0.3691857 ],
       [-0.30142432],
       [-0.08652845],
       [ 0.10284191],
       [ 0.28164166],
       [ 0.04798526],
       [ 0.30651772],
       [-0.44862366],
       [ 0.1269998 ],
       [ 0.07528514],
       [-0.0610241 ],
       [ 0.16293591],
       [ 0.37725383],
       [-0.02035037],
       [ 0.09504509],
       [-0.25850153],
       [-0.40130383],
       [ 0.35154438],
       [-0.453604  ],
       [-0.09244302],
       [ 0.45489007],
       [-0.1785185 ]], dtype=float32), array([0.], dtype=float32)]
before gan.train_on_batch()

 gan_model.train_on_batch dis last layer weight
modelName : discriminator last layer weights :
[array([[ 0.4115125 ],
       [ 0.0590893 ],
       [ 0.3830902 ],
       [ 0.3691857 ],
       [-0.3014

In [ ]:
xxx

In [ ]:
# evaluate the discriminator and plot real and fake points
def summarizePerformance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generateRealSamples(n)
    
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    
    # prepare fake examples
    x_fake, y_fake = generateFakeSamples(generator, latent_dim, n)
    
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    
    # scatter plot real and fake data points
    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    
    
    # save plot to file
    filename = imgOut + '/generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generateRealSamples(half_batch)
        
        # prepare fake examples
        x_fake, y_fake = generateFakeSamples(g_model, latent_dim, half_batch)
        
        # update discriminator
        # train_on_batch : Runs a single gradient update on a single batch of data
        d_model.train_on_batch(x_real, y_real) # AEDWIP syreturn_dict=False
        d_model.train_on_batch(x_fake, y_fake) # AEDWIP return_dict=False
        
        # prepare points in latent space as input for the generator
        x_gan = generateLatentPoints(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        # we want the discriminator to think the fakes are real
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan) # AEDWIP return_dict=False
        
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            summarizePerformance(i, g_model, d_model, latent_dim)

In [ ]:
%%time
# size of the latent space
latent_dim = 5

# create the discriminator
discriminator = defineDiscriminator()
discriminator.summary()

# # create the generator
generator = defineGenerator(latent_dim)
generator.summary()

# # create the gan
# gan_model = defineGan(generator, discriminator)

# # train model
# train(generator, discriminator, gan_model, latent_dim)

In [ ]:
4.83717840e-01 * 4.83717840e-01

In [ ]:
0.05250786  * 0.05250786 